In [4]:
import arxiv
import re
import pandas as pd
from data_utils import clean_data, clean_authors

In [5]:
def format_query(author='',title='',cat='',abstract=''):
    """Returns a formatted arxiv query string to handle
    simple queries of at most one instance each of these fields.
    To leave a field unspecified, leave the corresponding argument blank.
    
    e.g. format_query(cat='math.AP') will return
    the string used to pull all articles
    with the subject tag 'PDEs',
    since Math.AP is the subject tag
    for 'Analysis of PDEs'.

    Args:
        author: string to search for in the author field.
        title: string to search for in the title field.
        cat: A valid arxiv subject tag. See the full list of these at:
        https://arxiv.org/category_taxonomy
        abstract: string to search for in the abstract field.

    Returns:
        properly formatted query string to return
        all results simultaneously matching all specified fields.
    """

    tags = [f'au:{author}', f'ti:{title}', f'cat:{cat}', f'abs:{abstract}'] 
    # the tag.endswith(':') below
    # is for filtering out tags that
    # we do not pass to the function
    query = ' AND '.join([tag for tag in tags if not tag.endswith(':')])
    return query



def query_to_df(query,max_results):
    """Returns the results of an arxiv API query in a pandas dataframe.

    Args:
        query: string defining an arxiv query
        formatted according to 
        https://info.arxiv.org/help/api/user-manual.html#51-details-of-query-construction
        
        max_results: positive integer specifying
        the maximum number of results returned.

    Returns:
        pandas dataframe with one column for
        indivial piece of metadata of a returned result.
        To see a list of these columns and their descriptions,
        see the documentation for the
        Results class of the arxiv package here:
        http://lukasschwab.me/arxiv.py/index.html#Result

        The 'links' column is dropped and
        the authors column is replaced by
        a single string of each author name
        separated by a comma.

    """
    search = arxiv.Search(
            query = query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.LastUpdatedDate
            )
    results = search.results()

    drop_cols = ['authors','links','_raw']
    df = pd.DataFrame()

    for result in results:
        row_dict = {k : v for (k,v) in vars(result).items() if k not in drop_cols}
        row_dict['authors'] = ','.join([author.name for author in result.authors])
        row = pd.Series(row_dict)
        df = pd.concat([df , row.to_frame().transpose()], axis = 0)

    return df.reset_index(drop=True,inplace=False)


In [ ]:
def extract_date(date) :
    date_str = str(date)
    return round(float(date_str.split('-')[0])+float(date_str.split('-')[1])/12,2)

def get_category(category_str):
    return category_str[-2:].lower()

def get_categories(category_list):
    return [get_category(category_str) for category_str in category_list if '.' in category_str]

def get_authors(authors_str) :
    return [author for author in clean_authors(authors_str).split(',')]

## USER INTEREST SETS

Here we can each list ArXiv paper IDs to be used as inputs to the model to test its recommendations. See the example `ethan` below, which are Ethan's papers of interest.

Note that there is no need to restrict our interest sets to particular dates, or to not intersect the library we are recommending from (i.e., our dataset that we are pulling from).

In [6]:
ethan = ['1802.03426', '2304.14481', '2303.03190', '2210.13418',
         '2210.12824', '2210.00661', '2007.02390', '1808.05860',
         '2005.12732','1804.05690']
jeeuhn = ['0905.0486', 'math/0006187', '2106.07444', '1402.0490', 
          '1512.08942', '1603.09235', 'math/0510265', 'math/0505056', 
          'math/0604379', '2209.02568']
mike = ['2207.13571','2207.13498','2211.09644','2001.10647',
        '2103.08093','2207.08245', '2207.01677','2205.08744',
        '2008.04406','1912.09845']

In [ ]:
df.updated = df.updated.apply(extract_date)
df.published = df.published.apply(extract_date)
df.title = df.title.apply(clean_data)
df.summary = df.summary.apply(clean_data)
df.primary_category = df.primary_category.apply(get_category)
df.categories = df.categories.apply(get_categories)
df.authors = df.authors.apply(get_authors)